# Mga Generative na Network

Ang Recurrent Neural Networks (RNNs) at ang mga gated cell na variant nito tulad ng Long Short Term Memory Cells (LSTMs) at Gated Recurrent Units (GRUs) ay nagbibigay ng mekanismo para sa pagmomodelo ng wika, ibig sabihin, kaya nilang matutunan ang pagkakasunod-sunod ng mga salita at magbigay ng prediksyon para sa susunod na salita sa isang sequence. Dahil dito, magagamit natin ang RNNs para sa **mga generative na gawain**, tulad ng karaniwang pagbuo ng teksto, pagsasalin ng makina, at maging sa paglalagay ng caption sa mga imahe.

Sa arkitektura ng RNN na tinalakay natin sa nakaraang unit, ang bawat RNN unit ay gumagawa ng susunod na hidden state bilang output. Gayunpaman, maaari rin tayong magdagdag ng isa pang output sa bawat recurrent unit, na magpapahintulot sa atin na mag-output ng isang **sequence** (na may parehong haba sa orihinal na sequence). Bukod dito, maaari tayong gumamit ng mga RNN unit na hindi tumatanggap ng input sa bawat hakbang, at sa halip ay gumagamit lamang ng isang initial state vector, at pagkatapos ay gumagawa ng isang sequence ng mga output.

Sa notebook na ito, magpo-focus tayo sa mga simpleng generative na modelo na tumutulong sa atin na bumuo ng teksto. Para sa pagiging simple, magtatayo tayo ng **character-level network**, na bumubuo ng teksto letra kada letra. Sa panahon ng training, kailangan nating kumuha ng isang text corpus, at hatiin ito sa mga sequence ng letra.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Pagbuo ng Bokabularyo ng mga Karakter

Upang makabuo ng character-level generative network, kailangan nating hatiin ang teksto sa mga indibidwal na karakter sa halip na mga salita. Ang `TextVectorization` layer na ginagamit natin dati ay hindi kayang gawin ito, kaya may dalawang opsyon tayo:

* Manu-manong i-load ang teksto at gawin ang tokenization 'nang mano-mano', tulad ng nasa [opisyal na halimbawa ng Keras na ito](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Gamitin ang `Tokenizer` class para sa character-level tokenization.

Pipiliin natin ang pangalawang opsyon. Ang `Tokenizer` ay maaari ring gamitin para sa tokenization ng mga salita, kaya madali kang makakapagpalit mula sa char-level patungo sa word-level tokenization.

Upang gawin ang character-level tokenization, kailangan nating ipasa ang parameter na `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Gusto rin naming gumamit ng isang espesyal na token upang tukuyin ang **katapusan ng sequence**, na tatawagin naming `<eos>`. Idagdag natin ito nang manu-mano sa bokabularyo:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Pagsasanay ng isang generative RNN para lumikha ng mga pamagat

Ang paraan ng pagsasanay natin sa RNN upang lumikha ng mga pamagat ng balita ay ganito. Sa bawat hakbang, kukuha tayo ng isang pamagat, na ipapasok sa isang RNN, at para sa bawat input na karakter, hihilingin natin sa network na lumikha ng susunod na output na karakter:

![Larawan na nagpapakita ng halimbawa ng RNN na lumilikha ng salitang 'HELLO'.](../../../../../translated_images/tl/rnn-generate.56c54afb52f9781d.webp)

Para sa huling karakter ng ating sequence, hihilingin natin sa network na lumikha ng `<eos>` token.

Ang pangunahing pagkakaiba ng generative RNN na ginagamit natin dito ay kukunin natin ang output mula sa bawat hakbang ng RNN, at hindi lamang mula sa huling cell. Magagawa ito sa pamamagitan ng pag-specify ng `return_sequences` na parameter sa RNN cell.

Kaya, sa panahon ng pagsasanay, ang input sa network ay magiging isang sequence ng mga encoded na karakter na may tiyak na haba, at ang output ay magiging isang sequence na may parehong haba, ngunit naka-shift ng isang elemento at nagtatapos sa `<eos>`. Ang minibatch ay binubuo ng ilang ganitong mga sequence, at kakailanganin nating gumamit ng **padding** upang ma-align ang lahat ng mga sequence.

Gumawa tayo ng mga function na magbabago sa dataset para sa atin. Dahil gusto nating magpad ng mga sequence sa antas ng minibatch, una nating ibabatch ang dataset sa pamamagitan ng pagtawag sa `.batch()`, at pagkatapos ay `map` ito upang gawin ang transformation. Kaya, ang transformation function ay kukuha ng isang buong minibatch bilang parameter:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Ilang mahahalagang bagay na ginagawa natin dito:  
* Una, kinukuha natin ang aktwal na teksto mula sa string tensor  
* Ang `text_to_sequences` ay nagko-convert ng listahan ng mga string sa isang listahan ng mga integer tensor  
* Ang `pad_sequences` ay nagdadagdag ng padding sa mga tensor na ito hanggang sa kanilang pinakamahabang haba  
* Sa huli, ini-encode natin sa one-hot ang lahat ng mga karakter, at ginagawa rin ang shifting at pagdaragdag ng `<eos>`. Malalaman natin sa lalong madaling panahon kung bakit kailangan natin ng one-hot-encoded na mga karakter  

Gayunpaman, ang function na ito ay **Pythonic**, ibig sabihin, hindi ito maaaring awtomatikong ma-convert sa Tensorflow computational graph. Magkakaroon tayo ng mga error kung susubukan nating gamitin ang function na ito nang direkta sa `Dataset.map` function. Kailangan nating balutin ang tawag na ito gamit ang `py_function` wrapper:  


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note**: Ang pagkakaiba sa pagitan ng Pythonic at Tensorflow transformation functions ay maaaring mukhang masyadong komplikado, at maaaring iniisip mo kung bakit hindi natin ginagamit ang mga standard na Python functions para i-transform ang dataset bago ito ipasa sa `fit`. Bagama't posible itong gawin, ang paggamit ng `Dataset.map` ay may malaking bentahe, dahil ang data transformation pipeline ay isinasagawa gamit ang Tensorflow computational graph, na gumagamit ng GPU computations, at binabawasan ang pangangailangan na magpalipat-lipat ng data sa pagitan ng CPU/GPU.

Ngayon, maaari na nating buuin ang ating generator network at simulan ang training. Maaari itong basehan ng anumang recurrent cell na tinalakay natin sa nakaraang unit (simple, LSTM o GRU). Sa ating halimbawa, gagamit tayo ng LSTM.

Dahil ang network ay tumatanggap ng mga character bilang input, at maliit lang ang laki ng vocabulary, hindi natin kailangan ng embedding layer; ang one-hot-encoded input ay maaaring direktang ipasa sa LSTM cell. Ang output layer ay magiging isang `Dense` classifier na magko-convert ng LSTM output sa one-hot-encoded token numbers.

Bukod pa rito, dahil ang ating tinatrabaho ay mga sequence na may variable na haba, maaari tayong gumamit ng `Masking` layer upang lumikha ng mask na magpapawalang-bisa sa padded na bahagi ng string. Hindi ito mahigpit na kinakailangan, dahil hindi naman tayo masyadong interesado sa anumang bahagi na lampas sa `<eos>` token, ngunit gagamitin natin ito para magkaroon ng karanasan sa ganitong uri ng layer. Ang `input_shape` ay magiging `(None, vocab_size)`, kung saan ang `None` ay nagpapahiwatig ng sequence na may variable na haba, at ang output shape ay `(None, vocab_size)` din, tulad ng makikita mo sa `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Pagbuo ng Output

Ngayon na natapos na natin ang pagsasanay sa modelo, gusto nating gamitin ito upang makabuo ng output. Una sa lahat, kailangan natin ng paraan upang i-decode ang teksto na kinakatawan ng isang sequence ng mga numero ng token. Para magawa ito, maaari nating gamitin ang function na `tokenizer.sequences_to_texts`; gayunpaman, hindi ito mahusay gumagana sa character-level tokenization. Kaya't gagamit tayo ng dictionary ng mga token mula sa tokenizer (tinatawag na `word_index`), gagawa ng reverse map, at isusulat ang sarili nating decoding function:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Ngayon, magsisimula tayo sa isang string na `start`, i-encode ito sa isang sequence na `inp`, at pagkatapos sa bawat hakbang ay tatawagin natin ang ating network upang mahinuha ang susunod na karakter.

Ang output ng network na `out` ay isang vector na may `vocab_size` na mga elemento na kumakatawan sa mga probabilidad ng bawat token, at maaari nating mahanap ang pinaka-malamang na numero ng token gamit ang `argmax`. Pagkatapos, idinadagdag natin ang karakter na ito sa listahan ng mga na-generate na token, at nagpapatuloy sa pagbuo. Ang prosesong ito ng pagbuo ng isang karakter ay inuulit nang `size` na beses upang makabuo ng kinakailangang bilang ng mga karakter, at natatapos nang maaga kapag naabot ang `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Pagkuha ng sample ng output habang nagsasanay

Dahil wala tayong anumang kapaki-pakinabang na sukatan tulad ng *accuracy*, ang tanging paraan upang makita natin kung gumaganda ang ating modelo ay sa pamamagitan ng **pagkuha ng sample** ng mga nabuong string habang nagsasanay. Upang gawin ito, gagamit tayo ng **callbacks**, ibig sabihin, mga function na maaari nating ipasa sa `fit` function, at tatawagin ito nang pana-panahon habang nagsasanay.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Ang halimbawang ito ay nakabubuo na ng medyo maayos na teksto, ngunit maaari pa itong mapahusay sa iba't ibang paraan:

* **Mas maraming teksto**. Gumamit lamang tayo ng mga pamagat para sa ating gawain, ngunit maaari mong subukang gumamit ng buong teksto. Tandaan na hindi gaanong mahusay ang RNNs sa paghawak ng mahahabang mga sequence, kaya makatuwiran na hatiin ang mga ito sa mas maiikling pangungusap, o palaging magsanay gamit ang nakapirming haba ng sequence na may paunang itinakdang halaga `num_chars` (halimbawa, 256). Maaari mong subukang baguhin ang halimbawa sa itaas sa ganitong arkitektura, gamit ang [opisyal na tutorial ng Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) bilang inspirasyon.

* **Multilayer LSTM**. Makatuwiran ding subukan ang 2 o 3 layer ng mga LSTM cell. Gaya ng nabanggit natin sa nakaraang yunit, ang bawat layer ng LSTM ay kumukuha ng partikular na mga pattern mula sa teksto, at sa kaso ng character-level generator, maaari nating asahan na ang mas mababang antas ng LSTM ay responsable sa pagkuha ng mga pantig, at ang mas mataas na antas - para sa mga salita at kombinasyon ng mga salita. Madali itong maipatupad sa pamamagitan ng pagpapasa ng bilang ng mga layer bilang parameter sa LSTM constructor.

* Maaari mo ring subukang mag-eksperimento gamit ang **GRU units** at tingnan kung alin ang mas mahusay ang pagganap, pati na rin ang **iba't ibang laki ng hidden layer**. Ang sobrang laki ng hidden layer ay maaaring magresulta sa overfitting (halimbawa, matututunan ng network ang eksaktong teksto), at ang mas maliit na laki ay maaaring hindi makabuo ng magandang resulta.


## Malambot na pagbuo ng teksto at temperatura

Sa nakaraang depinisyon ng `generate`, palagi nating pinipili ang karakter na may pinakamataas na posibilidad bilang susunod na karakter sa nabubuong teksto. Dahil dito, madalas na "umuulit" ang teksto sa parehong mga sequence ng karakter nang paulit-ulit, tulad ng sa halimbawang ito:
```
today of the second the company and a second the company ...
```

Gayunpaman, kung titingnan natin ang distribusyon ng posibilidad para sa susunod na karakter, maaaring ang pagkakaiba sa pagitan ng ilang pinakamataas na posibilidad ay hindi ganoon kalaki, halimbawa, ang isang karakter ay maaaring may posibilidad na 0.2, habang ang isa pa ay 0.19, at iba pa. Halimbawa, kapag hinahanap ang susunod na karakter sa sequence na '*play*', ang susunod na karakter ay maaaring maging space o **e** (tulad ng sa salitang *player*).

Ito ay nagdadala sa atin sa konklusyon na hindi palaging "makatarungan" na piliin ang karakter na may mas mataas na posibilidad, dahil ang pagpili sa pangalawang pinakamataas ay maaari pa ring magresulta sa makabuluhang teksto. Mas matalino na **mag-sample** ng mga karakter mula sa distribusyon ng posibilidad na ibinigay ng output ng network.

Ang pag-sample na ito ay maaaring gawin gamit ang `np.multinomial` na function na nag-iimplementa ng tinatawag na **multinomial distribution**. Ang isang function na nag-iimplementa ng ganitong **malambot** na pagbuo ng teksto ay nakasaad sa ibaba:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Nagpakilala kami ng isa pang parameter na tinatawag na **temperature**, na ginagamit upang ipakita kung gaano kahigpit dapat tayong sumunod sa pinakamataas na posibilidad. Kung ang temperature ay 1.0, gumagawa tayo ng patas na multinomial sampling, at kapag ang temperature ay umabot sa infinity - nagiging pantay ang lahat ng posibilidad, at random nating pinipili ang susunod na karakter. Sa halimbawa sa ibaba, makikita natin na nagiging walang kahulugan ang teksto kapag masyadong tinaasan ang temperature, at nagiging katulad ng "cycled" na mahirap-generate na teksto kapag mas malapit ito sa 0.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
